In [ ]:
import os
import pickle
import tqdm
import cv2
import re
import json
import pylab as pl

from detectron2.evaluation import COCOEvaluator
from detectron2.data.datasets import load_coco_json, register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.data import DatasetMapper, transforms, build_detection_train_loader
from detectron2.engine import DefaultTrainer, SimpleTrainer
from detectron2.engine.hooks import PeriodicWriter
from detectron2.engine import DefaultPredictor

# Build CV model

In [ ]:
# Custom Trainer class to include custom data augmentations
class Trainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        # Set up data augmentation
        augs = [transforms.ResizeShortestEdge(
                    [640, 672, 704, 736, 768, 800],
                     max_size=1333, sample_style="choice"),
                transforms.RandomBrightness(0.8, 1.2),
                transforms.RandomSaturation(0.8, 1.2),
                transforms.RandomFlip(prob=0.5)]
        data_loader = build_detection_train_loader(cfg,
            mapper=DatasetMapper(cfg, is_train=True, 
                                 augmentations=augs))
        return data_loader

In [ ]:
def build_test_dict(datadir, image_dir, ann_file):
    # Test dataset
    name = ann_file.split('.')[0]
    test_dataset_name = f"test_{name}_data"
    test_data_dir = os.path.join(datadir, image_dir)
    test_json_file = os.path.join(datadir, image_dir, ann_file)

    register_coco_instances(test_dataset_name, {}, test_json_file, test_data_dir)
    test_dict = load_coco_json(test_json_file, test_data_dir,
                    dataset_name=test_dataset_name)
    test_metadata = MetadataCatalog.get(test_dataset_name)
    
    return test_dict, test_metadata

In [ ]:
def build_model(datadir, training_dict, training_metadata, test_dict, test_metadata, output_path):
    
    ######################
    # Model configuration
    
    # Most importantly, let's set up a model type
    model_type = "retinanet"
    model_file = "COCO-Detection/retinanet_R_50_FPN_3x.yaml"

    # Create a configuration and set up the model and datasets
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(model_file))
    cfg.DATASETS.TRAIN = (training_metadata.name,)
    cfg.DATASETS.TEST = (test_metadata.name,)
    cfg.OUTPUT_DIR = f"{output_path}/{model_type}_training_output"
    cfg.DATALOADER.NUM_WORKERS = 4
    if model_type == "maskrcnn":
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(training_metadata.thing_classes)
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05
    if model_type == "retinanet":
        cfg.MODEL.RETINANET.NUM_CLASSES = len(training_metadata.thing_classes)
        cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.05
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_file)  # Initialize weights from Model Zoo
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # (default: 512)

    # Solver options
    cfg.SOLVER.BASE_LR = 1e-3           # Base learning rate
    cfg.SOLVER.GAMMA = 0.5              # Learning rate decay
    cfg.SOLVER.STEPS = (250, 500, 750)  # Iterations at which to decay learning rate
    cfg.SOLVER.MAX_ITER = 1000          # Maximum number of iterations
    cfg.SOLVER.WARMUP_ITERS = 100       # Warmup iterations to linearly ramp learning rate from zero
    cfg.SOLVER.IMS_PER_BATCH = 1        # Lower to reduce memory usage (1 is the lowest)
    
    # Set low threshold (for high recall)
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.05
    
    ######################
    # Train Model
    
    # Create an output folder and delete any old files
    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    for fname in os.listdir(cfg.OUTPUT_DIR):
        if "tfevents" in fname:
            os.remove(os.path.join(cfg.OUTPUT_DIR, fname))

    # Create a Trainer using the data augmentations defined above and train the network
    # To avoid creating a custom trainer class, you can use 
    #  `trainer = SimpleTrainer(cfg)` : A no-frills training pipeline with nothing added  
    #  `trainer = DefaultTrainer(cfg)` : The default training pipeline with some data augmentation and hooks
    trainer = Trainer(cfg)

    # Hack to reduce the printing frequency, which defaults to every 20 iterations..
    # There is a cleaner way to do this using `trainer.build_hooks()` but it requires much more effort.
    for hook in trainer._hooks:
        if isinstance(hook, PeriodicWriter):
            hook._period = 50

    # Finally, train the network
    trainer.resume_or_load(resume=False)
    trainer.train()
     
    
    ######################
    # Evaluate model
    
    # Load weights from the most recent training run
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")

    # Run the official Detectron2 test evaluator, which returns useful metrics
    if model_type == "maskrcnn":
        eval_tasks = ("segm",)
    elif model_type == "retinanet":
        eval_tasks = ("bbox",)
    evaluator = COCOEvaluator(test_metadata.name,
                              tasks=eval_tasks,
                              distributed=False,
                              output_dir=f"{output_path}/{model_type}_test_output")
    trainer.test(cfg, trainer.model, evaluators=evaluator)

    
    return cfg

#cfg = build_model(datapath, training_dict, training_metadata, test_dict, test_metadata)

In [ ]:
datadir = '/crimea/wboag/2021/aclum/camera-detection/data/all_known'

train_dict, train_metadata = build_test_dict(datadir, 'non-bh', 'non-bh.json')
#train_dict, train_metadata = build_test_dict(datadir, 'bh', 'bh.json')
test_dict , test_metadata  = build_test_dict('data', 'nonresidential-all', 'foo.json')

output_path = 'output/detector2/nonresidential-1'
cfg = build_model(datadir, train_dict, train_metadata, test_dict, test_metadata, output_path)



# Run Model on All Images

In [ ]:
# Load metadata about the existing images
metadata_pkl = 'gsv/metadata.pkl'
if os.path.exists(metadata_pkl):
    with open(metadata_pkl, 'rb') as f:
        metadata = pickle.load(f)
else:
    metadata = {}
    
print(len(metadata))

In [ ]:
id_to_obj = {}
for sample in test_dict:
    id_to_obj[sample['image_id']] = sample
    
print(len(id_to_obj))

In [ ]:
predictor = DefaultPredictor(cfg)

predictions = {}
modelname = output_path.split('/')[-1]
with open('data/predictions/{modelname}.tsv', 'w') as f:
    print('File\tLatitude\tLongitude\tmax_prob\tnum_boxes\tboxes\tprobs\tclasses\tmanual_confirm', file=f)
    for i,sample in tqdm.tqdm(enumerate(test_dict)):
        image_id = sample['image_id']

        # Predict for image
        path = sample['file_name']
        filename = path.split('/')[-1]
    
        # Display location
        image_num = int(re.search('image0*(\d+)\.jpg', filename).groups()[0])
        gps = metadata[image_num]['location']

        sample = id_to_obj[image_id]
        path = sample['file_name']    
        img = cv2.imread(path)

        # How many predictions did the model make?
        instances = predictor(img)['instances']
        num_boxes = len(instances)
        probs   = json.dumps(instances.scores.tolist())
        boxes   = json.dumps([b.tolist() for b in instances.pred_boxes])
        classes = json.dumps(instances.pred_classes.tolist())
        score = max(probs)
        
        lat,long = map(float,gps.split(','))

        print(f'{filename}\t{lat}\t{long}\t{score}\t{num_boxes}\t{boxes}\t{probs}\t{classes}\t', file=f)
        #print(f'{filename},{gps},{score},{num_boxes},')

        
        if len(instances.scores):
            max_score = float(max(instances.scores))
            predictions[filename] = (max_score, image_id)
        
    #if i>= 10: break

In [ ]:
for i,(filename,(score,image_id)) in enumerate(sorted(predictions.items(), key=lambda t:t[1][0], reverse=True)):
    print(i)
    print(filename, score)
    
    # Display location
    image_num = int(re.search('image0*(\d+)\.jpg', filename).groups()[0])
    gps = metadata[image_num]['location']
    print(gps)

    sample = id_to_obj[image_id]
    path = sample['file_name']    
    img = cv2.imread(path)
    
    ################
    # Show Image
    pl.figure(figsize=(34,18))
    pl.imshow(img[:, :, ::-1])
        
    ################
    # Predicted Boxes
    instances = predictor(img)['instances']
    probs   = instances.scores 
    boxes   = instances.pred_boxes 
    classes = instances.pred_classes

    #colors = ['r', 'deepskyblue', 'b', 'g', 'c', 'm', 'y', 'tab:orange', 'tab:pink', 'goldenrod']
    colors = ['red', 'yellow']
    counter = 0
    for j,box in sorted(enumerate(boxes), key=lambda t:probs[t[0]], reverse=True):
        # Only show the 7 highest-probability boxes
        counter += 1
        if counter > 7: break
            
        #color = 'r' # colors[j]
        pred_label = classes[j]
        color = colors[pred_label]
        prob = probs[j]

        # Plot rectangle
        x1,y1,x2,y2 = box.tolist()
        rectangle = pl.Rectangle((x1,y1), x2-x1, y2-y1, ec=color, facecolor='None', lw=1+10*prob)
        pl.gca().add_patch(rectangle)

        # Plot probability
        pl.text(x1, y1*.99, f'{int(prob*100)}%', c=color, fontsize=6+50*prob)
            
    pl.show()
    
    if i>= 500: break